In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys

In [3]:
chunk1 = h5py.File(r'C:\Users\krish\Documents\spring23\DM\project\chunk1\chunk1.hdf5', 'r')

In [4]:
#earthquake
chunk2 = h5py.File(r'C:\Users\krish\Documents\spring23\DM\project\chunk2\chunk2.hdf5', 'r')

In [2]:
# print(sys.getsizeof(chunk1))

In [7]:
df_eq = pd.read_csv(r'C:\Users\krish\Documents\spring23\DM\project\chunk2\chunk2.csv')
df_noise = pd.read_csv(r'C:\Users\krish\Documents\spring23\DM\project\chunk1\chunk1.csv')


C:\Users\krish\AppData\Local\Temp\ipykernel_9596\3484923570.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eq = pd.read_csv(r'C:\Users\krish\Documents\spring23\DM\project\chunk2\chunk2.csv')


In [4]:
print(len(df_eq))
print(len(df_noise))

200000
235426


In [5]:
print(sys.getsizeof(df_eq))
print(sys.getsizeof(df_noise))

296982809
148181154


In [8]:
import random

data_array = []
# labels = np.array([]) # 1: earth-quake, 0: Non-earth-quake

# first we  will add earthquakes and then we will add non earthquakes

n_samples = 10000 # num of samples in each class

ev_list = df_eq['trace_name'].iloc[:n_samples].to_list()
lab1 = [1 for _ in range(len(ev_list))]
ev_list_noise = df_noise['trace_name'].iloc[:n_samples].to_list()
lab0 = [0 for _ in range(len(ev_list_noise))]

del df_eq,df_noise

ev_data = np.array(ev_list + ev_list_noise)
labels = np.array(lab1 + lab0)
del ev_list_noise,ev_list
del lab0,lab1

In [10]:
print(sys.getsizeof(labels))

80112


In [9]:
shuffled_index = np.random.permutation(len(labels))

ev_data = ev_data[shuffled_index]
labels = labels[shuffled_index]

In [10]:
print(ev_data[0])
print(ev_data[1])

AC06.C1_20180116162124_NO
AAM1.ZQ_20070619172908_EV


In [12]:
d = np.array(chunk2.get('data/'+str('ANGG.DK_20140910062331_EV')))
d.shape

(6000, 3)

In [71]:
# taking 3 channels
for c, evi in enumerate(ev_data):
    dataset = chunk1.get('data/'+str(evi)) 
    if dataset is None: # means that the wave is not in chunk2
        dataset = chunk2.get('data/'+str(evi))
        if dataset is None: # if it is not there even in chunk2, something's wrong
            print("the earthquake is not there in both the chunks")
            break
    data = np.array(dataset)
    data_array.append(data)
    if c%1000 == 0:
        print(c)

# for c, evi in enumerate(ev_list_noise):
#     dataset = chunk1.get('data/'+str(evi)) 
#     data = np.array(dataset)
#     data_array.append(data)
#     labels.append(0)
#     if c%1000 == 0:
#         print(5000+c)
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [87]:
# taking only 1 channel
data_array = []
for c, evi in enumerate(ev_data):
    dataset = chunk1.get('data/'+str(evi)) 
    if dataset is None: # means that the wave is not in chunk2
        dataset = chunk2.get('data/'+str(evi))
        if dataset is None: # if it is not there even in chunk2, something's wrong
            print("the earthquake is not there in both the chunks")
            break
    data = np.array(dataset)
    data_array.append(data[:,2])
    if c%1000 == 0:
        print(c)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [72]:
for arr in data_array:
    if len(arr) == 0:
        print('empty data')
        break
    else:
        pass

In [ ]:
!conda install -c tensorflow

In [73]:
import tensorflow as tf
from tensorflow.keras import layers

class InceptionBlock(tf.keras.Model):
    def __init__(self, n_filters):
        super(InceptionBlock, self).__init__()
        self.conv1 = layers.Conv1D(filters=n_filters//4, kernel_size=1, activation='relu')
        self.conv2 = layers.Conv1D(filters=n_filters//4, kernel_size=2, padding='same', activation='relu')
        self.conv3 = layers.Conv1D(filters=n_filters//4, kernel_size=3, padding='same', activation='relu')
        self.maxpool = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')
        self.concat = layers.Concatenate(axis=-1)
        
    def call(self, inputs):
        x1 = self.conv1(inputs)
        x2 = self.conv2(inputs)
        x3 = self.conv3(inputs)
        x4 = self.maxpool(inputs)
        x4 = self.conv1(x4)
        x = self.concat([x1, x2, x3, x4])
        return x


def build_inception_time(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    
    x = inputs
    x = layers.Conv1D(filters=16, kernel_size=1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # Inception Blocks
    x = InceptionBlock(n_filters=16)(x)
    x = InceptionBlock(n_filters=16)(x)
    x = layers.Dropout(0.5)(x)
    
    x = InceptionBlock(n_filters=32)(x)
    x = InceptionBlock(n_filters=32)(x)
    x = layers.Dropout(0.5)(x)
    
    x = InceptionBlock(n_filters=64)(x)
    x = InceptionBlock(n_filters=64)(x)
    x = layers.Dropout(0.5)(x)
    
    x = InceptionBlock(n_filters=128)(x)
    x = InceptionBlock(n_filters=128)(x)
    x = layers.Dropout(0.5)(x)
    
    x = layers.GlobalAveragePooling1D()(x)
    
    # Append LSTM layers with dropout and kernel regularization
    x = layers.Reshape((-1, 1))(x)
    x = layers.LSTM(32, return_sequences=True, kernel_regularizer=regularizers.L2(0.01))(x)
    x = layers.LSTM(16, kernel_regularizer=regularizers.L2(0.01))(x)
    # Dense layers with kernel regularization
    x = layers.Dense(8, activation='relu', kernel_regularizer=regularizers.L2(0.01))(x)
    x = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L2(0.01))(x)

    outputs = x

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model

model = build_inception_time((6000,3),2)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 6000, 3)]         0         
                                                                 
 conv1d_433 (Conv1D)         (None, 6000, 16)          64        
                                                                 
 batch_normalization_19 (Bat  (None, 6000, 16)         64        
 chNormalization)                                                
                                                                 
 inception_block_138 (Incept  (None, 6000, 16)         396       
 ionBlock)                                                       
                                                                 
 inception_block_139 (Incept  (None, 6000, 16)         396       
 ionBlock)                                                       
                                                          

In [97]:
# import tensorflow as tf
# from tensorflow.keras import layers

# class InceptionBlock(tf.keras.Model):
#     def __init__(self, n_filters):
#         super(InceptionBlock, self).__init__()
#         self.conv1 = layers.Conv1D(filters=n_filters//4, kernel_size=1, activation='relu')
#         self.conv2 = layers.Conv1D(filters=n_filters//4, kernel_size=2, padding='same', activation='relu')
#         self.conv3 = layers.Conv1D(filters=n_filters//4, kernel_size=3, padding='same', activation='relu')
#         self.maxpool = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')
#         self.concat = layers.Concatenate(axis=-1)
        
#     def call(self, inputs):
#         x1 = self.conv1(inputs)
#         x2 = self.conv2(inputs)
#         x3 = self.conv3(inputs)
#         x4 = self.maxpool(inputs)
#         x4 = self.conv1(x4)
#         x = self.concat([x1, x2, x3, x4])
#         return x


# def build_inception_time(input_shape, num_classes):
#     inputs = layers.Input(shape=input_shape)
    
#     x = inputs
#     x = layers.Conv1D(filters=16, kernel_size=1, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
    
#     # Inception Blocks
#     x = InceptionBlock(n_filters=16)(x)
#     x = InceptionBlock(n_filters=16)(x)
#     x = layers.Dropout(0.5)(x)
    
#     x = InceptionBlock(n_filters=32)(x)
#     x = InceptionBlock(n_filters=32)(x)
#     x = layers.Dropout(0.5)(x)
    
#     x = InceptionBlock(n_filters=64)(x)
#     x = InceptionBlock(n_filters=64)(x)
#     x = layers.Dropout(0.5)(x)
    
#     x = InceptionBlock(n_filters=128)(x)
#     x = InceptionBlock(n_filters=128)(x)
#     x = layers.Dropout(0.5)(x)
    
#     x = layers.GlobalAveragePooling1D()(x)
    
#     # Append LSTM layers with dropout and kernel regularization
# #     x = layers.Reshape((-1, 1))(x)
# #     x = layers.LSTM(32, return_sequences=True, kernel_regularizer=regularizers.L2(0.01))(x)
# #     x = layers.LSTM(16, kernel_regularizer=regularizers.L2(0.01))(x)
# #     # Dense layers with kernel regularization
#     x = layers.Dense(8, activation='relu', kernel_regularizer=regularizers.L2(0.01))(x)
#     x = layers.Dense(1, activation='sigmoid', kernel_regularizer=regularizers.L2(0.01))(x)

#     outputs = x

#     model = tf.keras.Model(inputs=inputs, outputs=outputs)

#     return model

# model = build_inception_time((6000,1),2)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()


Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 6000, 1)]         0         
                                                                 
 conv1d_508 (Conv1D)         (None, 6000, 16)          32        
                                                                 
 batch_normalization_22 (Bat  (None, 6000, 16)         64        
 chNormalization)                                                
                                                                 
 inception_block_162 (Incept  (None, 6000, 16)         396       
 ionBlock)                                                       
                                                                 
 inception_block_163 (Incept  (None, 6000, 16)         396       
 ionBlock)                                                       
                                                          

In [89]:
data_array = np.array(data_array)

In [90]:
type(data_array)

numpy.ndarray

In [91]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(data_array, labels, test_size=0.20, random_state=42)

# Split the train set into train and validation sets (75/25 split)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Print the shapes of the resulting sets
from keras.utils import to_categorical

# y_train = to_categorical(y_train)
# # y_val = to_categorical(y_val)
# y_test = to_categorical(y_test)


# print("Train set shape:", X_train.shape, y_train.shape)
# # print("Validation set shape:", X_val.shape, y_val.shape)
# print("Test set shape:", X_test.shape, y_test.shape)


In [92]:
X_train.shape

(16000, 6000)

In [198]:
# # model 2 -> only lstm
# # model 1 is overfitting
# # model 3 -> resnet+lstm

# import tensorflow as tf

# # Define the model
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(128, input_shape=(6000, 1), return_sequences=True),
#     tf.keras.layers.LSTM(64),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

# # Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print the model summary
# model.summary()


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_64 (LSTM)              (None, 6000, 128)         66560     
                                                                 
 lstm_65 (LSTM)              (None, 64)                49408     
                                                                 
 dense_58 (Dense)            (None, 32)                2080      
                                                                 
 dense_59 (Dense)            (None, 1)                 33        
                                                                 
Total params: 118,081
Trainable params: 118,081
Non-trainable params: 0
_________________________________________________________________


In [108]:
X_train.shape

X_train = X_train[:6000]

In [109]:
y_train.shape

y_train = y_train[:6000]

In [110]:
import tensorflow as tf
# Fit the model with the training data and validation data, using the callbacks above
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split = 0.25)

Epoch 1/10
36/36 [==============================] - 245s 7s/step - loss: 0.5424 - accuracy: 0.7320 - val_loss: 0.5549 - val_accuracy: 0.7265
Epoch 2/10
36/36 [==============================] - 244s 7s/step - loss: 0.5351 - accuracy: 0.7364 - val_loss: 0.5375 - val_accuracy: 0.7315
Epoch 3/10
36/36 [==============================] - 252s 7s/step - loss: 0.5261 - accuracy: 0.7489 - val_loss: 0.5350 - val_accuracy: 0.7340
Epoch 4/10
36/36 [==============================] - 239s 7s/step - loss: 0.5246 - accuracy: 0.7476 - val_loss: 0.5327 - val_accuracy: 0.7415
Epoch 5/10
36/36 [==============================] - 246s 7s/step - loss: 0.5336 - accuracy: 0.7540 - val_loss: 0.5484 - val_accuracy: 0.7385
Epoch 6/10
36/36 [==============================] - 244s 7s/step - loss: 0.5321 - accuracy: 0.7587 - val_loss: 0.5419 - val_accuracy: 0.7347
Epoch 7/10
36/36 [==============================] - 245s 7s/step - loss: 0.5407 - accuracy: 0.7489 - val_loss: 0.5684 - val_accuracy: 0.7097
Epoch 8/10
27

KeyboardInterrupt: 

In [203]:
list(X_train[0]) 

[228.51093,
 -180.73,
 -424.5082,
 -327.3679,
 -107.59573,
 -205.28378,
 -396.99615,
 -465.14868,
 -388.9923,
 -267.12415,
 -29.413174,
 357.05817,
 684.4956,
 741.7545,
 441.869,
 166.55591,
 27.044628,
 -162.5188,
 -448.63828,
 -596.61835,
 -318.12683,
 23.658625,
 100.403854,
 243.99673,
 682.6902,
 998.16394,
 798.7407,
 347.3873,
 12.474083,
 -212.8688,
 -309.0427,
 -447.12952,
 -717.2902,
 -803.0724,
 -661.2502,
 -265.92117,
 301.93173,
 687.1757,
 541.1918,
 128.11444,
 180.76814,
 457.23233,
 284.01846,
 -360.1883,
 -897.1766,
 -796.514,
 -292.26526,
 160.16064,
 409.24835,
 427.37534,
 383.25098,
 225.87384,
 -253.35126,
 -658.5386,
 -771.21674,
 -842.973,
 -845.1921,
 -380.8527,
 436.5021,
 943.81903,
 996.0153,
 631.3009,
 97.32016,
 -171.71582,
 -140.64337,
 116.299225,
 422.3902,
 546.86865,
 402.87247,
 15.718728,
 -284.37228,
 -267.63818,
 -129.2012,
 98.03771,
 291.81412,
 128.63068,
 -262.71317,
 -378.12283,
 -145.68964,
 0.1483383,
 12.182203,
 -39.121716,
 -229.26886

In [ ]:
for ele in X_train[:10]:
    print(len(ele))

In [111]:
from sklearn.metrics import classification_report, confusion_matrix

# Assuming you have a trained model and test data stored in X_test and y_test variables
y_pred = model.predict(X_test)

y_pred_ = []

for ele in y_pred:
    if ele>0.5:
        y_pred_.append(1)
    else:
        y_pred_.append(0)
# Print confusion matrix

print("Confusion Matrix:")
print(confusion_matrix(y_test[:2500], y_pred_[:2500]))

# Print classification report
print("Classification Report:")
print(classification_report(y_test[:2500], y_pred_[:2500]))


125/125 [==============================] - 37s 293ms/step
Confusion Matrix:
[[1142   60]
 [ 601  697]]
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.95      0.78      1202
           1       0.92      0.54      0.68      1298

    accuracy                           0.74      2500
   macro avg       0.79      0.74      0.73      2500
weighted avg       0.79      0.74      0.73      2500

